In [72]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

loader = PyPDFLoader("Zain Ali Shah.pdf")
data = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = splitter.split_documents(data)

# Embeding
db = Chroma.from_documents(documents, OpenAIEmbeddings())
# export db as retriver
retriever = db.as_retriever()


In [73]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F6C79502F0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F6C7952210>, root_client=<openai.OpenAI object at 0x000001F6C79510D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F6C7950320>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [74]:
# Design Prompt Template
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. Think step by step before providing a detailed answer.
I will tip you $1000 if the user is satisfied with your answer.

Context:
{context}

Question: {input}
""")

prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context. Think step by step before providing a detailed answer.\nI will tip you $1000 if the user is satisfied with your answer.\n\nContext:\n{context}\n\nQuestion: {input}\n'), additional_kwargs={})])

In [75]:
# Chains
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [78]:
# Retriver chain
from langchain.chains.retrieval import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever=retriever,combine_docs_chain=document_chain)

user_input = input("I'm a zain assistant, Ask whatever about zain: ")
if user_input:
    response = retriever_chain.invoke({"input": user_input})
    print(response["answer"])

Based on the provided context, Syed Zain Ali Shah has the following work experience:

1. **Frontend Engineer at Kodexo Labs** 
   - **Duration**: January 2025 - Present
   - **Location**: Pakistan
   - **Responsibilities**:
     - Worked on building and supporting AI applications to help clients solve real-world business problems with innovative solutions.
     - Developed multiple AI-driven projects while continuously learning and implementing new AI tools and frameworks.
     - Collaborated with cross-functional teams to deliver high-quality, efficient, and customized AI solutions for diverse client needs.

2. **Full Stack Developer at Techsol LLC**
   - **Duration**: January 2024 - December 2024
   - **Location**: Pakistan

These roles highlight Zain's focus on developing AI applications and full-stack development, as well as his involvement in learning and adapting new tools and frameworks related to his field.
